In [1]:
import pandas as pd

In [3]:
dfOvertime = pd.read_csv("bleuOvertime.csv")

In [8]:
pd.set_option('display.max_colwidth', None)

In [11]:
dfOvertime = dfOvertime.dropna()
dfOvertime

,Unnamed: 0,Question,GT Answer,GT Topic,2014 Predicted,2014 Topic,2016 Bleu Score,2016 Predicted,2016 Topic,2018 Bleu Score,2018 Predicted,2018 Topic,2020 Bleu Score,2020 Predicted,2020 Topic,max - min
0,0,Which NFL team represented the AFC at Super Bowl 50?,['Denver Broncos'],Super_Bowl_50,Denver,{'Denver': 'NFL International Series'},50.000000,Denver Broncos,Super Bowl 50,0.0,New England Patriots,2017–18 NFL playoffs,100.0,New England Patriots,Super Bowl LII,100.000000
1,1,Which NFL team represented the NFC at Super Bowl 50?,['Carolina Panthers'],Super_Bowl_50,Philadelphia Eagles,{'Philadelphia Eagles': 'Super Bowl XXXIX'},0.000000,New York Giants,Super Bowl,100.0,Philadelphia Eagles,Super Bowl XXXIX,100.0,Philadelphia Eagles,2017–18 NFL playoffs,100.000000
6,6,What day was the game played on?,"['February 7, 2016', 'February 7']",Super_Bowl_50,"December 18, 1932","{'December 18, 1932': '1932 NFL Playoff Game'}",30.326533,December 29,1963 NFL Championship Game,100.0,December 29,1963 NFL Championship Game,100.0,December 29,1963 NFL Championship Game,69.673467
7,7,What is the AFC short for?,['American Football Conference'],Super_Bowl_50,American Football Conference,{'American Football Conference': 'List of AFC champions'},100.000000,American Football Conference,AFC East,100.0,American Football Conference,AFC East,100.0,American Football Conference,List of AFC champions,0.000000
8,8,What does AFC stand for?,['American Football Conference'],Super_Bowl_50,Asian Football Confederation,{'Asian Football Confederation': 'Asian Football Confederation'},100.000000,Asian Football Confederation,Asian Football Confederation,-1.0,None,None,100.0,Asian Football Confederation,Asian Football Confederation,101.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10518,10518,What is another word for centripetal force?,"['radial', 'radial (centripetal) force']",Force,radial,{'radial': 'Force'},100.000000,radial,Force,100.0,radial,Force,100.0,radial,Force,0.000000
10528,10528,"Tension, compression, and drag are what kind of forces?","['Nonconservative', 'Nonconservative forces other than friction', 'Nonconservative forces']",Force,Nonconservative,{'Nonconservative': 'Force'},100.000000,Nonconservative,Force,100.0,Nonconservative,Force,100.0,Nonconservative,Force,0.000000
10531,10531,What is the exchange of heat associated with?,['nonconservative forces'],Force,EM radiation is emitted and then absorbed,{'EM radiation is emitted and then absorbed': 'Thermal radiation'},-1.000000,None,None,0.0,lifting force,Convection,-1.0,None,None,1.000000
10533,10533,What makes energy changes in a closed system?,['nonconservative forces'],Force,heat transfer,{'heat transfer': 'Internal energy'},100.000000,heat transfer,Internal energy,100.0,heat transfer,Internal energy,-1.0,None,None,101.000000


In [20]:
dfOvertime = dfOvertime.mask(dfOvertime.eq('None')).dropna()

In [21]:
dfOvertime.sort_values("max - min").tail(100).sample(10)

,Unnamed: 0,Question,GT Answer,GT Topic,2014 Predicted,2014 Topic,2016 Bleu Score,2016 Predicted,2016 Topic,2018 Bleu Score,2018 Predicted,2018 Topic,2020 Bleu Score,2020 Predicted,2020 Topic,max - min
4733,4733,When were the public housing developments built in the neighborhood?,['between the 1960s and 1990s'],"Fresno,_California",1936-37,{'1936-37': 'Harlem River Houses'},100.0,1936-37,Harlem River Houses,0.0,1938,Trumbull Park Homes,0.0,1954,Lowden Homes,100.0
4144,4144,How many votes in total does the Council have?,['352'],European_Union_law,51,{'51': 'New York City Council'},0.0,26,New York City Council,100.0,51,New York City Council,100.0,26,New York City Council,100.0
3912,3912,For about how long would the extended LM allow a surface stay on the moon?,"['3 days', '3 days.', 'over 3 days']",Apollo_program,ten minutes,{'ten minutes': 'Apollo 16'},0.0,three-day,Apollo 16,100.0,three-day,Apollo 16,100.0,three-day,Apollo 16,100.0
4299,4299,How many species of trees can be found in the Amazon rainforest?,"['16,000']",Amazon_rainforest,"100,000","{'100,000': 'Amazon insects'}",100.0,"100,000",Amazon insects,0.0,"16,000",Amazon rainforest,100.0,"16,000",Amazon rainforest,100.0
3986,3986,From what pad was Apollo 5 launched from?,"['pad 37', '37']",Apollo_program,"Wallops Island, Virginia","{'Wallops Island, Virginia': 'Little Joe 5'}",0.0,LC-39A,Apollo 4,100.0,LC-39A,Apollo 4,100.0,"Wallops Island, Virginia",Little Joe 5,100.0
4402,4402,How many invertebrate species are known in Brazil alone?,"['96,660 and 128,843', 'between 96,660 and 128,843', '128,843']",Amazon_rainforest,270,{'270': 'List of birds of Brazil'},100.0,270,List of birds of Brazil,0.0,232,List of birds of Brazil,0.0,238,List of birds of Brazil,100.0
4407,4407,What amount of bird species on earth are found in the Amazon rainforest?,"['One in five', '1,294', 'One in five of all the bird species in the world']",Amazon_rainforest,"1,622","{'1,622': 'Wildlife of Brazil'}",100.0,"1,622",Wildlife of Brazil,0.0,One in five,Amazon rainforest,100.0,One in five,Amazon rainforest,100.0
4740,4740,How many seats does the amphitheatre at Woodward park have?,"['2,500']","Fresno,_California","12,500","{'12,500': 'DTE Energy Music Theatre'}",100.0,"12,500",DTE Energy Music Theatre,0.0,"17,500",Lakeview Amphitheater,0.0,"6,400",Park Theater (Las Vegas),100.0
3989,3989,"What was the nickname for the test where, during abort mode, the ascent engine was started and fired?","['""fire-in-the-hole""', 'fire-in-the-hole']",Apollo_program,Pad Abort Test 1,{'Pad Abort Test 1': 'Pad Abort Test 1'},0.0,Redundant Set Launch Sequencer,Space Shuttle abort modes,100.0,Pad Abort Test 1,Pad Abort Test 1,100.0,Pad Abort Test 1,Pad Abort Test 1,100.0
3999,3999,What day did the Apollo 11 crew return to Earth?,"['July 24', 'July 24.']",Apollo_program,July 24,{'July 24': 'Apollo 11'},100.0,July 24,Apollo 11,100.0,July 24,Apollo 11,0.0,April 13,STS-41-C,100.0


In [24]:
dfGT = pd.read_csv("bleuGT.csv")
dfGT = dfGT.dropna()
dfGT = dfGT.mask(dfGT.eq('None')).dropna()

In [26]:
dfGT.sort_values("max - min").tail(100).sample(10)

,Unnamed: 0,Question,GT Answer,GT Topic,2014 Bleu Score,2014 Predicted,2014 Topic,2016 Bleu Score,2016 Predicted,2016 Topic,2018 Bleu Score,2018 Predicted,2018 Topic,2020 Bleu Score,2020 Predicted,2020 Topic,max - min
4001,4001,Which lunar probe was near the Apollo 12 crew's landing site?,['Surveyor 3'],Apollo_program,100.0,Surveyor 3,Apollo 12,0.0,Discovery,STS-48,100.0,Surveyor 3,Apollo 12,100.0,Surveyor 3,Apollo 12,100.0
5678,5678,What company owns the American Broadcasting Company?,"['The Walt Disney Company', 'Disney–ABC Television Group']",American_Broadcasting_Company,0.0,George Storer,NBC,0.0,CA. Noble,NBC,0.0,CBS,NBC,100.0,Walt Disney Television,List of programs broadcast by American Broadcasting Company,100.0
4033,4033,When was Skylab's last crew on the station before it re-entered Earth's atmosphere?,"['February 8, 1974', 'February 8, 1974,']",Apollo_program,100.0,"February 8, 1974",Skylab,100.0,"February 8, 1974",Skylab,0.0,1979,Skylab mutiny,100.0,"February 8, 1974",Skylab,100.0
5684,5684,What kind of network was ABC when it first began?,"['radio network', 'radio']",American_Broadcasting_Company,100.0,radio,American Broadcasting Company,0.0,commercial broadcast television,List of programs broadcast by American Broadcasting Company,0.0,commercial broadcast television,List of programs broadcast by American Broadcasting Company,0.0,commercial broadcasting television,List of programs broadcast by American Broadcasting Company,100.0
5679,5679,In what year did ABC stylize it's logo as abc?,['1957'],American_Broadcasting_Company,0.0,1962,American Broadcasting Company,100.0,1957,American Broadcasting Company,100.0,1957,American Broadcasting Company,0.0,1962,Circle 7 logo,100.0
5535,5535,What type of works from Asia are included in the V&A's British galleries?,"['porcelain, cloth and wallpaper']",Victoria_and_Albert_Museum,0.0,historical artifacts,British Museum Department of Asia,0.0,historical artifacts,British Museum Department of Asia,100.0,"porcelain, cloth and wallpaper",Victoria and Albert Museum,100.0,"porcelain, cloth and wallpaper",Victoria and Albert Museum,100.0
4027,4027,"How much was the Apollo project estimated to have cost from 1959 to 1973, the length of the program?","['$20.4 billion', '20.4 billion,', '20.4 billion']",Apollo_program,100.0,$20.4 billion,Apollo program,100.0,$20.4 billion,Apollo program,0.0,$20,Apollo program,50.0,$25.4 billion,Budget of NASA,100.0
3901,3901,What type of antenna was used for communication on the lunar flights?,"['high-gain S-band antenna', 'high-gain S-band', 'S-band']",Apollo_program,100.0,high-gain,Earth-Moon-Earth communication,100.0,high-gain,Earth–Moon–Earth communication,0.0,VHF scimitar,Scimitar antenna,0.0,parabolic,Communication Moon Relay,100.0
5348,5348,How many universities does Newcastle have?,['two'],Newcastle_upon_Tyne,0.0,24,Newcastle University,100.0,two,Newcastle upon Tyne,0.0,38,University of Newcastle (Australia),100.0,two,Newcastle upon Tyne,100.0
5934,5934,What is the native format for the Walt Disney Company's US TV properties?,"['720p high definition', '480i']",American_Broadcasting_Company,0.0,24/7,AWE (TV network),0.0,Chile,Univision,0.0,1080i,TV Land,100.0,720p high definition,American Broadcasting Company,100.0


In [27]:
dfGT

,Unnamed: 0,Question,GT Answer,GT Topic,2014 Bleu Score,2014 Predicted,2014 Topic,2016 Bleu Score,2016 Predicted,2016 Topic,2018 Bleu Score,2018 Predicted,2018 Topic,2020 Bleu Score,2020 Predicted,2020 Topic,max - min
0,0,Which NFL team represented the AFC at Super Bowl 50?,['Denver Broncos'],Super_Bowl_50,36.787944,Denver,NFL International Series,100.000000,Denver Broncos,Super Bowl 50,0.0,New England Patriots,2017–18 NFL playoffs,0.000000,New England Patriots,Super Bowl LII,100.0
1,1,Which NFL team represented the NFC at Super Bowl 50?,['Carolina Panthers'],Super_Bowl_50,0.000000,Philadelphia Eagles,Super Bowl XXXIX,0.000000,New York Giants,Super Bowl,0.0,Philadelphia Eagles,Super Bowl XXXIX,0.000000,Philadelphia Eagles,2017–18 NFL playoffs,0.0
6,6,What day was the game played on?,"['February 7, 2016', 'February 7']",Super_Bowl_50,0.000000,"December 18, 1932",1932 NFL Playoff Game,0.000000,December 29,1963 NFL Championship Game,0.0,December 29,1963 NFL Championship Game,0.000000,December 29,1963 NFL Championship Game,0.0
7,7,What is the AFC short for?,['American Football Conference'],Super_Bowl_50,100.000000,American Football Conference,List of AFC champions,100.000000,American Football Conference,AFC East,100.0,American Football Conference,AFC East,100.000000,American Football Conference,List of AFC champions,0.0
9,9,What day was the Super Bowl played on?,"['February 7, 2016', 'February 7']",Super_Bowl_50,0.000000,"January 17, 1971",Super Bowl V,0.000000,"January 12, 1975",Super Bowl IX,0.0,"January 28, 2001",Super Bowl,0.000000,"January 12, 1975",Super Bowl IX,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10506,10506,What may a force on one part of an object affect?,"['other parts', 'other parts of an object']",Force,100.000000,other parts of an object,Force,100.000000,other parts of an object,Force,100.0,other parts of an object,Force,100.000000,other parts of an object,Force,0.0
10517,10517,What force changes an objects direction of travel?,"['centripetal', 'unbalanced centripetal force']",Force,0.000000,radial,Force,33.333333,radial (centripetal) force,Force,50.0,Coriolis force,Coriolis force,33.333333,radial (centripetal) force,Force,50.0
10518,10518,What is another word for centripetal force?,"['radial', 'radial (centripetal) force']",Force,100.000000,radial,Force,100.000000,radial,Force,100.0,radial,Force,100.000000,radial,Force,0.0
10528,10528,"Tension, compression, and drag are what kind of forces?","['Nonconservative', 'Nonconservative forces other than friction', 'Nonconservative forces']",Force,100.000000,Nonconservative,Force,100.000000,Nonconservative,Force,100.0,Nonconservative,Force,100.000000,Nonconservative,Force,0.0
